In [1]:
!pip install transformers nltk vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 9.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import json
import glob
import os
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

Mounted at /content/drive


In [3]:
df = pd.read_csv("data/responses.csv")
df.head()

,age,gender,politics,religion,prompt,response,model,temperature,response_filled,is_blank
0,15,female,democrat,atheist,art,Art profoundly shapes society and culture by r...,grok,1.5,Art profoundly shapes society and culture by r...,False
1,50,non-binary,republican,atheist,art,Art profoundly shapes society and culture by r...,grok,1.5,Art profoundly shapes society and culture by r...,False
2,50,female,republican,atheist,art,Art profoundly influences society and culture ...,grok,1.5,Art profoundly influences society and culture ...,False
3,25,male,democrat,atheist,art,Art profoundly influences society and culture ...,grok,1.5,Art profoundly influences society and culture ...,False
4,15,non-binary,republican,atheist,art,Art profoundly influences society and culture ...,grok,1.5,Art profoundly influences society and culture ...,False


## POLITENESS

In [4]:
def load_classifier(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    clf = pipeline("text-classification", model=model, tokenizer=tokenizer, truncation=True)
    return clf

In [5]:
politeness_clf = load_classifier("Intel/polite-guard")

def compute_politeness(text):
    out = politeness_clf(text)[0]
    return out["label"], out["score"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Device set to use cuda:0


In [6]:
df["politeness_label"], df["politeness_score"] = zip(
    *df["response"].apply(compute_politeness)
)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [8]:
def create_politeness_table(df, characteristic, metric="politeness_score"):
    if metric not in df.columns:
        raise ValueError(f"Metric '{metric}' not found. Run compute_politeness first.")

    models = sorted(df['model'].unique())
    categories = sorted(df[characteristic].unique())

    results = []

    for model in models:
        model_df = df[df['model'] == model]
        row = {'Model': model}

        if metric == "politeness_score":
            # Add politeness score for each category
            for category in categories:
                score = model_df[model_df[characteristic] == category][metric].mean()
                row[str(category)] = score

            # Add average across all categories
            row['Average'] = model_df[metric].mean()

        elif metric == "politeness_label":
            # For labels, show proportion of "POLITE" responses
            for category in categories:
                cat_data = model_df[model_df[characteristic] == category]
                polite_prop = (cat_data[metric] == 'POLITE').mean()
                row[str(category)] = polite_prop

            # Add average proportion
            row['Average'] = (model_df[metric] == 'POLITE').mean()

        results.append(row)

    results_df = pd.DataFrame(results)

    # Round numeric columns
    numeric_cols = results_df.select_dtypes(include=[np.number]).columns
    results_df[numeric_cols] = results_df[numeric_cols].round(4)

    return results_df


def export_all_politeness_tables_html(df, output_file='politeness_tables.html', metric='politeness_score'):
    characteristics = ['age', 'gender', 'religion', 'politics']

    # Define color schemes for each characteristic
    colors = {
        'age': '#3498db',       # Blue
        'gender': '#e74c3c',    # Red
        'religion': '#9b59b6',  # Purple
        'politics': '#f39c12'   # Orange
    }

    # Metric display name
    if metric == 'politeness_score':
        metric_name = 'Politeness Score'
    else:
        metric_name = 'Politeness (% Polite)'

    html_content = f"""
    <html>
    <head>
        <style>
            body {{
                font-family: Arial, sans-serif;
                margin: 20px;
            }}
            table {{
                border-collapse: collapse;
                margin-bottom: 30px;
                width: auto;
            }}
            th, td {{
                border: 1px solid #ddd;
                padding: 8px 12px;
                text-align: center;
            }}
            tr:nth-child(even) {{
                background-color: #f9f9f9;
            }}
            tr:hover:not(.title-row) {{
                background-color: #f0f0f0;
            }}
            .model-col {{
                text-align: left;
                font-weight: bold;
            }}

            /* Title row styling */
            .title-row {{
                background-color: #2c3e50;
                color: white;
                font-size: 16px;
                font-weight: bold;
            }}

            /* Model header - always gray */
            .model-header {{
                background-color: #5d6d7e;
                color: white;
                font-weight: bold;
            }}

            /* Age headers - blue */
            .age-header {{
                background-color: #3498db;
                color: white;
                font-weight: bold;
            }}

            /* Gender headers - red */
            .gender-header {{
                background-color: #e74c3c;
                color: white;
                font-weight: bold;
            }}

            /* Religion headers - purple */
            .religion-header {{
                background-color: #9b59b6;
                color: white;
                font-weight: bold;
            }}

            /* Politics headers - orange */
            .politics-header {{
                background-color: #f39c12;
                color: white;
                font-weight: bold;
            }}
        </style>
    </head>
    <body>
        <h1>{metric_name} by Characteristic</h1>
    """

    for char in characteristics:
        char_df = create_politeness_table(df, char, metric)

        # Start table
        html_content += '<table>\n'

        # Add title row that spans all columns
        num_cols = len(char_df.columns)
        html_content += f'<thead>\n<tr class="title-row">\n<th colspan="{num_cols}">{metric_name} by {char.capitalize()}</th>\n</tr>\n'

        # Add column headers
        html_content += '<tr>\n'
        for col in char_df.columns:
            if col == 'Model':
                html_content += f'<th class="model-header">{col}</th>\n'
            else:
                html_content += f'<th class="{char}-header">{col}</th>\n'
        html_content += '</tr>\n</thead>\n'

        # Add data rows
        html_content += '<tbody>\n'
        for _, row in char_df.iterrows():
            html_content += '<tr>\n'
            for col, val in row.items():
                if col == 'Model':
                    html_content += f'<td class="model-col">{val}</td>\n'
                else:
                    html_content += f'<td>{val}</td>\n'
            html_content += '</tr>\n'

        html_content += '</tbody>\n</table>\n\n'

    html_content += """
    </body>
    </html>
    """

    # Save to file
    with open(output_file, 'w') as f:
        f.write(html_content)

    print(f"All tables saved to {output_file}")
    print("Open the file in a browser, then copy each table into Google Docs")


def export_individual_politeness_tables_html(df, metric='politeness_score'):
    characteristics = ['age', 'gender', 'religion', 'politics']

    colors = {
        'age': '#3498db',       # Blue
        'gender': '#e74c3c',    # Red
        'religion': '#9b59b6',  # Purple
        'politics': '#f39c12'   # Orange
    }

    if metric == 'politeness_score':
        metric_name = 'Politeness Score'
    else:
        metric_name = 'Politeness (% Polite)'

    for char in characteristics:
        char_df = create_politeness_table(df, char, metric)

        html_content = f"""
        <html>
        <head>
            <style>
                table {{
                    border-collapse: collapse;
                    margin: 20px;
                }}
                th, td {{
                    border: 1px solid #ddd;
                    padding: 8px 12px;
                    text-align: center;
                }}
                tr:nth-child(even) {{
                    background-color: #f9f9f9;
                }}
                tr:hover:not(.title-row) {{
                    background-color: #f0f0f0;
                }}
                .model-col {{
                    text-align: left;
                    font-weight: bold;
                }}
                .title-row {{
                    background-color: #2c3e50;
                    color: white;
                    font-size: 16px;
                    font-weight: bold;
                }}
                .model-header {{
                    background-color: #5d6d7e;
                    color: white;
                    font-weight: bold;
                }}
                .data-header {{
                    background-color: {colors[char]};
                    color: white;
                    font-weight: bold;
                }}
            </style>
        </head>
        <body>
        """

        # Build table with title row
        num_cols = len(char_df.columns)
        html_content += '<table>\n<thead>\n'
        html_content += f'<tr class="title-row">\n<th colspan="{num_cols}">{metric_name} by {char.capitalize()}</th>\n</tr>\n'

        # Column headers
        html_content += '<tr>\n'
        for col in char_df.columns:
            if col == 'Model':
                html_content += f'<th class="model-header">{col}</th>\n'
            else:
                html_content += f'<th class="data-header">{col}</th>\n'
        html_content += '</tr>\n</thead>\n'

        # Data rows
        html_content += '<tbody>\n'
        for _, row in char_df.iterrows():
            html_content += '<tr>\n'
            for col, val in row.items():
                if col == 'Model':
                    html_content += f'<td class="model-col">{val}</td>\n'
                else:
                    html_content += f'<td>{val}</td>\n'
            html_content += '</tr>\n'

        html_content += '</tbody>\n</table>\n</body>\n</html>'

        filename = f'politeness_by_{char}.html'
        with open(filename, 'w') as f:
            f.write(html_content)

        print(f"Table saved to {filename}")


export_all_politeness_tables_html(df, 'politeness_score_all_characteristics.html', metric='politeness_score')
export_all_politeness_tables_html(df, 'politeness_label_all_characteristics.html', metric='politeness_label')

All tables saved to politeness_score_all_characteristics.html
Open the file in a browser, then copy each table into Google Docs
All tables saved to politeness_label_all_characteristics.html
Open the file in a browser, then copy each table into Google Docs
